In [1]:
import LinearAlgebra
import LinearAlgebra.norm
import LinearAlgebra.I
import LinearAlgebra.svd
import LinearAlgebra.Diagonal

In [2]:
eye(n) = 1.0*Matrix(I,n,n)

eye (generic function with 1 method)

In [3]:
function householder(a)
    n = length(a)
    α = a[1]
    σ = norm(a, 2)
    β = 1 + abs(α) / σ
    u = zeros(n)
    u[1:end] = a[1:end] * (sign(α)/(σ + abs(α)))
    u[1] = 1.0
    return u, β, - sign(α) * σ
end

householder (generic function with 1 method)

In [4]:
function upper_bidiagonalization(A)
    m = size(A)[1]
    n = size(A)[2]
    V = eye(n)
    B = zeros(m, n)
    B[1:m, 1:n] = A[1:m, 1:n]
    U = eye(m)
    u, r, σ = householder(B[1:end, 1])
    s = length(u)
    Q = eye(m)
    Q[m-s+1:m, m-s+1:m] -= r * u * transpose(u)
    B = Q * B
    B[2:end, 1] = zeros(s-1)
    U = U * Q
    if n >= m
        for i = 2 : m
            u, r, σ = householder(B[i-1, i:end])
            s = length(u)
            P = eye(n)
            P[n-s+1:n, n-s+1:n] -= r * u * transpose(u)
            B = B * P
            B[i-1, i+1:end] = zeros(s-1)
            V = V * P
            
            u, r, σ = householder(B[i:end, i])
            s = length(u)
            Q = eye(m)
            Q[m-s+1:m, m-s+1:m] -= r * u * transpose(u)
            B = Q * B
            B[i+1:end, i] = zeros(s-1)
            U = U * Q
        end
        if n > m
            u, r, σ = householder(B[m, m+1:end])
            s = length(u)
            P = eye(n)
            P[n-s+1:n, n-s+1:n] -= r * u * transpose(u)
            B = B * P
            B[m, m+1+1:end] = zeros(s-1)
            V = V * P
        end
    else
        for i = 2 : n
            u, r, σ = householder(B[i-1, i:end])
            s = length(u)
            P = eye(n)
            P[n-s+1:n, n-s+1:n] -= r * u * transpose(u)
            B = B * P
            B[i-1, i+1:end] = zeros(s-1)
            V = V * P
            
            u, r, σ = householder(B[i:end, i])
            s = length(u)
            Q = eye(m)
            Q[m-s+1:m, m-s+1:m] -= r * u * transpose(u)
            B = Q * B
            B[i+1:end, i] = zeros(s-1)
            U = U * Q
        end
    end
    return U, B, transpose(V)
end

upper_bidiagonalization (generic function with 1 method)

In [5]:
function zero_non_diagonal(A)
    m = size(A)[1]
    n = size(A)[2]
    for i = 1 : m
        for j = 1 : n
            if i != j
                A[i, j] = 0
            end
        end
    end
end

zero_non_diagonal (generic function with 1 method)

In [6]:
function givens_rotation(f, g)
    c = 0.0
    s = 0.0
    r = 0.0
    if f == 0.0 
        c = 0.0
        s = 1.0
        r = g
    elseif abs(f) > abs(g)
        t = g / f
        t_1 = sqrt(1.0 + t * t)
        c = 1.0 / t_1
        s = t * c
        r = f * t_1
    else 
        t = f / g
        t_1 = sqrt(1.0 + t * t) 
        s = 1 / t_1
        c = t * s 
        r = g * t_1 
    end
    return c, s, r
end

givens_rotation (generic function with 1 method)

In [7]:
function zero_shift_qr_for_m_geq_n(A)
    m = size(A)[1]
    n = size(A)[2]
    B = zeros(m, n)
    B[1:m, 1:n] = A[1:m, 1:n]
    L = eye(m)
    P = eye(n)
#     Zamienić warunek stopu na osiągnięcie przez normę elementów poza przekątną wartości mniejszej niż
#     zadana precyzja? Mało dokładana implemetacja - może się nigdy nie skończyć.
    p = 100
    for k = 1 : 2*p
        for i = 1 : n - 1
            c, s, r = givens_rotation(B[i,i], B[i,i+1])

    #       % construct matrix Q and multiply on the right by Q^T
    #       % this annihilates both B(i-1,i+1) and B(i,i+1) 
    #       % but makes B(i+1,i) non-zero
            
            Q = eye(n)
            Q[i:i+1, i:i+1] = [c s; -s c]
            B = B * transpose(Q)
            P = P * transpose(Q)
            c, s, r = givens_rotation(B[i,i], B[i+1,i])

    #       % construct matrix Q and multiply on the left by Q
    #       % This annihilates B(i+1,i) but makes B(i,i+1) and
    #       % B(i,i+2) non-zero
            
            Q = eye(m)
            Q[i:i+1,i:i+1] = [c s; -s c]
            B = Q * B
            L = Q * L
#             Zerować elementy poza diagonalną i superdiagonalną?
#             Raczej nie, bo chociaż może to przyśpieszy zbieżność B, to jednak B rozjedzie nam się z L i P
        end
    end
    return transpose(L), B, transpose(P)
end

zero_shift_qr_for_m_geq_n (generic function with 1 method)

In [8]:
function absolute_eigenvalues(U, Σ)
    m = size(Σ)[1]
    n = size(Σ)[2]
    k = min(m, n)
    for i = 1 : k
        if Σ[i, i] < 0 
            Σ[i, i] = Σ[i, i] * (-1.0)
            for j = 1 : m
                U[j, i] = U[j, i] * (-1.0)
            end
        end
    end
end

absolute_eigenvalues (generic function with 1 method)

In [9]:
function singular_value_decomposition(A)
    m = size(A)[1]
    n = size(A)[2]
    if m >= n
        U, B, V_T = upper_bidiagonalization(A)
        L, Σ, P = zero_shift_qr_for_m_geq_n(B)
        zero_non_diagonal(Σ)
        X = zeros(size(U*L))
        X[1:m, 1:m] = (U*L)[1:m, 1:m]
        absolute_eigenvalues(X, Σ)
        return X, Σ, P*V_T
    else
        U, B, V_T = upper_bidiagonalization(transpose(A))
        L, Σ, P = zero_shift_qr_for_m_geq_n(B)
        precision = 1.0/(2.0^48)
        zero_non_diagonal(Σ)
        X = zeros(size(transpose(P*V_T)))
        X[1:m, 1:m] = (transpose(P*V_T))[1:m, 1:m]
        Σ_T = zeros(size(transpose(Σ)))
        Σ_T[1:m, 1:n] = (transpose(Σ))[1:m, 1:n]
        absolute_eigenvalues(X, Σ_T)
        return X, Σ_T, transpose(U*L)
    end
    
end

singular_value_decomposition (generic function with 1 method)

In [10]:
T = [1. 2. 3.; 4. 5. 6.; 7. 8. 9.; 10. 11. 12.; 13. 14. 15.]

5×3 Array{Float64,2}:
  1.0   2.0   3.0
  4.0   5.0   6.0
  7.0   8.0   9.0
 10.0  11.0  12.0
 13.0  14.0  15.0

In [11]:
U, D, V_T = singular_value_decomposition(T)

([-0.101346 0.767938 … -0.168619 0.346815; -0.248569 0.488071 … 0.18527 0.106011; … ; -0.543014 -0.0716624 … 0.781001 -0.0483299; -0.690237 -0.351529 … -0.466484 0.423985], [35.1826 0.0 0.0; 0.0 1.47691 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0], [-0.519273 -0.575521 -0.631769; -0.750792 -0.0459264 0.65894; -0.408248 0.816497 -0.408248])

In [16]:
U

5×5 Array{Float64,2}:
 -0.101346   0.767938   -0.501285   -0.168619   0.346815 
 -0.248569   0.488071    0.808973    0.18527    0.106011 
 -0.395792   0.208204   -0.062816   -0.331169  -0.828481 
 -0.543014  -0.0716624  -0.296144    0.781001  -0.0483299
 -0.690237  -0.351529    0.0512731  -0.466484   0.423985 

In [17]:
D

5×3 Array{Float64,2}:
 35.1826  0.0      0.0        
  0.0     1.47691  0.0        
  0.0     0.0      1.02302e-15
  0.0     0.0      0.0        
  0.0     0.0      0.0        

In [18]:
V_T

3×3 Array{Float64,2}:
 -0.519273  -0.575521   -0.631769
 -0.750792  -0.0459264   0.65894 
 -0.408248   0.816497   -0.408248

In [19]:
U*D*V_T

5×3 Array{Float64,2}:
  1.0   2.0   3.0
  4.0   5.0   6.0
  7.0   8.0   9.0
 10.0  11.0  12.0
 13.0  14.0  15.0

In [20]:
F = svd(T)

LinearAlgebra.SVD{Float64,Float64,Array{Float64,2}}([-0.101346 0.767938 -0.220084; -0.248569 0.488071 0.186841; … ; -0.543014 -0.0716624 0.774941; -0.690237 -0.351529 -0.514134], [35.1826, 1.47691, 4.15619e-16], [-0.519273 -0.575521 -0.631769; -0.750792 -0.0459264 0.65894; -0.408248 0.816497 -0.408248])

In [21]:
F.U

5×3 Array{Float64,2}:
 -0.101346   0.767938   -0.220084
 -0.248569   0.488071    0.186841
 -0.395792   0.208204   -0.227565
 -0.543014  -0.0716624   0.774941
 -0.690237  -0.351529   -0.514134

In [22]:
F.S

3-element Array{Float64,1}:
 35.182648331894214    
  1.476907699980088    
  4.156185784097889e-16

In [23]:
F.Vt

3×3 Array{Float64,2}:
 -0.519273  -0.575521   -0.631769
 -0.750792  -0.0459264   0.65894 
 -0.408248   0.816497   -0.408248

In [24]:
F.U * Diagonal(F.S) * F.Vt

5×3 Array{Float64,2}:
  1.0   2.0   3.0
  4.0   5.0   6.0
  7.0   8.0   9.0
 10.0  11.0  12.0
 13.0  14.0  15.0

In [25]:
X = rand(5,5)

5×5 Array{Float64,2}:
 0.643165  0.9303    0.113812  0.658829  0.487808
 0.339668  0.142551  0.263919  0.925643  0.216322
 0.39469   0.745577  0.903083  0.232201  0.259584
 0.996727  0.996432  0.41811   0.386727  0.354792
 0.43054   0.309442  0.195831  0.998699  0.985927

In [26]:
U, D, V_T = singular_value_decomposition(X)

([-0.492658 0.0333786 … -0.0208266 0.722574; -0.314119 -0.404352 … -0.797246 -0.00461684; … ; -0.530373 0.434344 … -0.148354 -0.620534; -0.476642 -0.643462 … 0.549569 -0.222487], [2.75298 0.0 … 0.0 0.0; 0.0 1.10359 … 0.0 0.0; … ; 0.0 0.0 … 0.435231 0.0; 0.0 0.0 … 0.0 0.231739], [-0.47597 -0.533225 … -0.503614 -0.387565; 0.208771 0.513544 … -0.647832 -0.386263; … ; -0.267864 0.0877584 … -0.491247 0.823586; -0.729248 0.602115 … 0.249954 -0.146804])

In [27]:
U

5×5 Array{Float64,2}:
 -0.492658   0.0333786  -0.48335    -0.0208266   0.722574  
 -0.314119  -0.404352    0.319695   -0.797246   -0.00461684
 -0.387472   0.482377    0.730717    0.199823    0.208091  
 -0.530373   0.434344   -0.350683   -0.148354   -0.620534  
 -0.476642  -0.643462    0.0851035   0.549569   -0.222487  

In [28]:
D

5×5 Array{Float64,2}:
 2.75298  0.0      0.0       0.0       0.0     
 0.0      1.10359  0.0       0.0       0.0     
 0.0      0.0      0.638066  0.0       0.0     
 0.0      0.0      0.0       0.435231  0.0     
 0.0      0.0      0.0       0.0       0.231739

In [29]:
V_T

5×5 Array{Float64,2}:
 -0.47597   -0.533225   -0.292042   -0.503614  -0.387565 
  0.208771   0.513544    0.351855   -0.647832  -0.386263 
 -0.355406  -0.285831    0.876558    0.151279  -0.0273583
 -0.267864   0.0877584   0.0304955  -0.491247   0.823586 
 -0.729248   0.602115   -0.147058    0.249954  -0.146804 

In [30]:
U*D*V_T

5×5 Array{Float64,2}:
 0.643165  0.9303    0.113812  0.658829  0.487808
 0.339668  0.142551  0.263919  0.925643  0.216322
 0.39469   0.745577  0.903083  0.232201  0.259584
 0.996727  0.996432  0.41811   0.386727  0.354792
 0.43054   0.309442  0.195831  0.998699  0.985927

In [37]:
X

5×5 Array{Float64,2}:
 0.643165  0.9303    0.113812  0.658829  0.487808
 0.339668  0.142551  0.263919  0.925643  0.216322
 0.39469   0.745577  0.903083  0.232201  0.259584
 0.996727  0.996432  0.41811   0.386727  0.354792
 0.43054   0.309442  0.195831  0.998699  0.985927

In [31]:
F = svd(X)

LinearAlgebra.SVD{Float64,Float64,Array{Float64,2}}([-0.492658 0.0333786 … 0.0208266 -0.722574; -0.314119 -0.404352 … 0.797246 0.00461684; … ; -0.530373 0.434344 … 0.148354 0.620534; -0.476642 -0.643462 … -0.549569 0.222487], [2.75298, 1.10359, 0.638066, 0.435231, 0.231739], [-0.47597 -0.533225 … -0.503614 -0.387565; 0.208771 0.513544 … -0.647832 -0.386263; … ; 0.267864 -0.0877584 … 0.491247 -0.823586; 0.729248 -0.602115 … -0.249954 0.146804])

In [32]:
F.U

5×5 Array{Float64,2}:
 -0.492658   0.0333786  -0.48335     0.0208266  -0.722574  
 -0.314119  -0.404352    0.319695    0.797246    0.00461684
 -0.387472   0.482377    0.730717   -0.199823   -0.208091  
 -0.530373   0.434344   -0.350683    0.148354    0.620534  
 -0.476642  -0.643462    0.0851035  -0.549569    0.222487  

In [33]:
F.S

5-element Array{Float64,1}:
 2.7529837812703097 
 1.1035870997241983 
 0.6380662043997454 
 0.43523085093227837
 0.23173900941131828

In [34]:
F.Vt

5×5 Array{Float64,2}:
 -0.47597   -0.533225   -0.292042   -0.503614  -0.387565 
  0.208771   0.513544    0.351855   -0.647832  -0.386263 
 -0.355406  -0.285831    0.876558    0.151279  -0.0273583
  0.267864  -0.0877584  -0.0304955   0.491247  -0.823586 
  0.729248  -0.602115    0.147058   -0.249954   0.146804 

In [35]:
F.U * Diagonal(F.S) * F.Vt

5×5 Array{Float64,2}:
 0.643165  0.9303    0.113812  0.658829  0.487808
 0.339668  0.142551  0.263919  0.925643  0.216322
 0.39469   0.745577  0.903083  0.232201  0.259584
 0.996727  0.996432  0.41811   0.386727  0.354792
 0.43054   0.309442  0.195831  0.998699  0.985927

In [36]:
X

5×5 Array{Float64,2}:
 0.643165  0.9303    0.113812  0.658829  0.487808
 0.339668  0.142551  0.263919  0.925643  0.216322
 0.39469   0.745577  0.903083  0.232201  0.259584
 0.996727  0.996432  0.41811   0.386727  0.354792
 0.43054   0.309442  0.195831  0.998699  0.985927

In [38]:
Y = zeros(size(transpose(T)))
Y[1:3, 1:5] = transpose(T)[1:3, 1:5]

3×5 Array{Float64,2}:
 1.0  4.0  7.0  10.0  13.0
 2.0  5.0  8.0  11.0  14.0
 3.0  6.0  9.0  12.0  15.0

In [39]:
U, D, V_T = singular_value_decomposition(Y)

([-0.519273 0.750792 0.408248; -0.575521 0.0459264 -0.816497; -0.631769 -0.65894 0.408248], [35.1826 0.0 … 0.0 0.0; 0.0 1.47691 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [-0.101346 -0.248569 … -0.543014 -0.690237; -0.767938 -0.488071 … 0.0716624 0.351529; … ; -0.168619 0.18527 … 0.781001 -0.466484; 0.346815 0.106011 … -0.0483299 0.423985])

In [40]:
U

3×3 Array{Float64,2}:
 -0.519273   0.750792    0.408248
 -0.575521   0.0459264  -0.816497
 -0.631769  -0.65894     0.408248

In [41]:
D

3×5 Array{Float64,2}:
 35.1826  0.0      0.0          0.0  0.0
  0.0     1.47691  0.0          0.0  0.0
  0.0     0.0      1.02302e-15  0.0  0.0

In [42]:
V_T

5×5 LinearAlgebra.Transpose{Float64,Array{Float64,2}}:
 -0.101346  -0.248569  -0.395792  -0.543014   -0.690237 
 -0.767938  -0.488071  -0.208204   0.0716624   0.351529 
  0.501285  -0.808973   0.062816   0.296144   -0.0512731
 -0.168619   0.18527   -0.331169   0.781001   -0.466484 
  0.346815   0.106011  -0.828481  -0.0483299   0.423985 

In [43]:
U*D*V_T

3×5 Array{Float64,2}:
 1.0  4.0  7.0  10.0  13.0
 2.0  5.0  8.0  11.0  14.0
 3.0  6.0  9.0  12.0  15.0